In [9]:
import numpy as np
import pandas as pd
import qgrid as qg
qg.set_grid_option('maxVisibleRows',10)

from quantopian.pipeline.data import EquityPricing
import matplotlib.pyplot as plt
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.domain import US_EQUITIES
from quantopian.pipeline.filters import QTradableStocksUS, AtLeastN
from quantopian.research import run_pipeline
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, BusinessDaysSincePreviousEvent, DailyReturns,MarketCap, MaxDrawdown

from quantopian.pipeline.data.factset import Fundamentals
from quantopian.pipeline.data.factset.estimates import PeriodicConsensus
import quantopian.pipeline.data.factset.estimates as fe

from quantopian.pipeline.data import EquityPricing as ep


In [15]:
def make_pipeline():
    # Create our pipeline  
    pipe = Pipeline()  
    
    stocks = morningstar.valuation.market_cap.latest.top(700)
    base_universe = QTradableStocksUS()
    
    #giving the slack of 65 days so it would not keep changing constantly
    universe=AtLeastN(inputs=[stocks],
                       window_length=70,
                       N=65,) & base_universe
    
    price= EquityPricing.close.latest        
        
    # Add the surprise factor to the pipeline.
    pipe = Pipeline(
        columns={
            'close': price,
          

        },
        domain=US_EQUITIES,
        screen=universe 
        
    )    
    days_since_earnings = BusinessDaysSincePreviousEvent(
                          inputs=[Fundamentals.eps_rpt_date_qf]
                        )
    
  
    pipe.add(days_since_earnings, "days_since_earnings")
    
    return pipe  


# Non parametric Jump detection

In [16]:
pipe=make_pipeline()
earn_p = run_pipeline(pipe, '2014-09-01', '2015-04')
earn_p = earn_p.loc[:,earn_p.notnull().all()]
qg.show_grid(earn_p['close'])


## computing jump detection statistic:L 
k   = 16
r = np.append(np.nan, np.diff(np.log(earn_p['close'])))
    
#calculate bipolar variation
bpv = np.multiply(np.absolute(r[:]), np.absolute(np.append(np.nan, r[:-1])))

#create a 2 dimensional array
bpv = np.append(np.nan, bpv[0:-1]).reshape(-1,1)
m= len(bpv) * [np.nan]


#Calculating L statistic

In [17]:
#the original paper has sqrt(sig)
for i in range(k-2, len(bpv)):
    m[i] = np.mean(bpv[i-k+2: i]) 

#volatility estimate
sig = m[i] 
L = r/sig

qg.show_grid(pd.DataFrame(L))

QgridWidget(grid_options={'syncColumnCellResize': True, 'highlightSelectedRow': True, 'editable': True, 'enabl…

#calculate T statistic 

In [18]:
## calculate T statistic for Significance test

n   = np.size(earn_p['close'])
c   = (2/np.pi)**0.5
Sn  = c*(2*np.log(n))**0.5
Cn  = (2*np.log(n))**0.5/c - np.log(np.pi*np.log(n))/(2*c*(2*np.log(n))**0.5)
       

#Jump threshold 
with Significance level of 1%
This rejects null hypothesis of no jump. 
B* for 1% is 4.066(for hard coding)

The J statistic gives the binary values as output, where 1 represents trade signal


The data frame can be filtered to see just how many trading signals(1) we have.
since there are too many signals, we need to add further filter with respect to normalised census data

In [19]:
# Jump threshold    
significance_level=0.01
beta_star   = -np.log(-np.log(1-significance_level)) 
T   = (abs(L)-Cn)*Sn
    
#Binary value is jump with direction with significance level(sign)
J   = (T > beta_star).astype(float)
# First k rows are NaN
#J[0:k] = np.nan
qg.show_grid(pd.DataFrame(J))

QgridWidget(grid_options={'syncColumnCellResize': True, 'highlightSelectedRow': True, 'editable': True, 'enabl…

##conditional merge

1. J statistic has to be 1
2. If log returns is Negative <- Take short position
3. If log returns is Positive <- Take long position

In [6]:
pipe=make_pipeline()

if J==1 and r<=0:
    u==-1
elif J==1 and r>0:
    d==1
    
    
pipe.add(J, "pin")
earn_p = run_pipeline(pipe, '2014-09-01', '2015-04')
earn_p = earn_p.loc[:,earn_p.notnull().all()]




ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()